<a href="https://colab.research.google.com/github/debojit11/nlp_practice/blob/main/Text_classification_with_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext

In [ ]:
!ls -lh /content/

total 14G
-rw-r--r-- 1 root root 6.8G Jan 18  2019 cc.en.300.bin
-rw-r--r-- 1 root root 6.5G Jan 18  2019 cc.hi.300.bin
-rw-r--r-- 1 root root  36M Mar 31 12:13 ecommerce_dataset.csv
-rw-r--r-- 1 root root 6.9M Mar 31 12:56 ecommerce.test
-rw-r--r-- 1 root root  28M Mar 31 12:56 ecommerce.train
drwxr-xr-x 1 root root 4.0K Mar 24 13:34 sample_data


In [ ]:
!wget https://raw.githubusercontent.com/codebasics/nlp-tutorials/refs/heads/main/18_fasttext_classification/ecommerce_dataset.csv

--2025-03-31 13:13:07--  https://raw.githubusercontent.com/codebasics/nlp-tutorials/refs/heads/main/18_fasttext_classification/ecommerce_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36898651 (35M) [text/plain]
Saving to: ‘ecommerce_dataset.csv.1’

ecommerce_dataset.c 100%[===================>]  35.19M  2.02MB/s    in 18s     

2025-03-31 13:13:26 (1.91 MB/s) - ‘ecommerce_dataset.csv.1’ saved [36898651/36898651]



In [ ]:
import pandas as pd
df = pd.read_csv("/content/ecommerce_dataset.csv", names=['category', 'description'], header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [ ]:
df.category.value_counts()

,count
category,
Household,19313
Books,11820
Electronics,10621
Clothing & Accessories,8671


In [ ]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [ ]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [ ]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)
df.category.unique()

<ipython-input-8-409ce1645d63>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)


array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [ ]:
df['category'] = '__label__' + df['category'].astype(str)
df.head()

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [ ]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [ ]:
df['category_description'][0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

In [ ]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r"[^\w\s']", " ", text)
text = re.sub(r" +", " ", text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [ ]:
preprocess("  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi")

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [ ]:
df['category_description'] = df['category_description'].apply(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [ ]:
from sklearn.model_selection import train_test_split
import fasttext

In [ ]:
train, test = train_test_split(df, test_size=0.2)
train.shape, test.shape

((40339, 3), (10085, 3))

In [ ]:
train.to_csv('ecommerce.train', columns=['category_description'], index=False, header=False)
test.to_csv('ecommerce.test', columns=['category_description'], index=False, header=False)

In [ ]:
model = fasttext.train_supervised(input='ecommerce.train')
model.test('ecommerce.test')

(10085, 0.9696579077838374, 0.9696579077838374)

In [ ]:
import numpy as np

In [ ]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99210846]))

In [ ]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000989]))

In [ ]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [ ]:
model.get_nearest_neighbors("sony")

[(0.9993773102760315, 'bullguard'),
 (0.9993767738342285, 'eyecups'),
 (0.9993751049041748, 'endurable'),
 (0.9993751049041748, 'ipd'),
 (0.9993751049041748, 'pupillary'),
 (0.9993751049041748, '420ft'),
 (0.9993739128112793, '8x40mm'),
 (0.9993646740913391, 'ursi'),
 (0.9993646740913391, 'bearberry'),
 (0.9993646740913391, 'paeonia')]

In [ ]:
model.get_nearest_neighbors("painting")

[(0.998810887336731, 'llp'),
 (0.9988036751747131, 'energizing'),
 (0.9988006353378296, 'copolymer'),
 (0.9987891912460327, 'emollients'),
 (0.9987786412239075, 'tms'),
 (0.998776912689209, 'borne'),
 (0.9987768530845642, 'rhpackage'),
 (0.9987768530845642, 'rhequipped'),
 (0.9987768530845642, '52x38x15mm'),
 (0.9987768530845642, '05x1')]

In [ ]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'girsl'),
 (0.0, 'mand'),
 (0.0, 'mudassir')]